In [1]:
# WikiText-103 is a large-scale language modeling dataset extracted from verified Good and Featured articles on Wikipedia
# # > 100M words across 29k articles
from datasets import load_dataset
import pandas as pd
wiki_dataset = load_dataset("yehzw/wikitext-103", "raw")
print(wiki_dataset)
_ = pd.DataFrame.from_dict(wiki_dataset["train"][:5])  
print(_.head())


/home/admin/_github/massimodipaolo/ai-crash-course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 29567
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 60
    })
    test: Dataset({
        features: ['text'],
        num_rows: 62
    })
})
                                                text
0  = Valkyria Chronicles III =\nSenjō no Valkyria...
1  = Tower Building of the Little Rock Arsenal =\...
2  = Cicely Mary Barker =\nCicely Mary Barker (28...
3  = Gambia women's national football team =\nThe...
4  = Plain maskray =\nThe plain maskray or brown ...


In [ ]:
from tokenizers import Tokenizer 
from tokenizers.models import BPE # BPE: Byte Pair Encoding model, subword tokenization
tokenizer = Tokenizer(BPE(unk_token="[UNK]")) # Initialize a BPE tokenizer with a specified unknown token

In [3]:
# trainer instance to train the BPE tokenizer
from tokenizers.trainers import BpeTrainer
# special tokens: unknown, class, separator, padding, mask
# unknown: used for out-of-vocabulary tokens
# class: used to denote the start of a sequence
# separator: used to separate different segments in a sequence
# padding: used to pad sequences to a uniform length, used in batching
# mask: used in masked language modeling tasks like BERT
# The order in which you write the special tokens list matters: here "[UNK]" will get the ID 0, "[CLS]" will get the ID 1...
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
print(trainer)

BpeTrainer(BpeTrainer(min_frequency=0, vocab_size=30000, show_progress=True, special_tokens=[AddedToken(content="[UNK]", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True), AddedToken(content="[CLS]", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True), AddedToken(content="[SEP]", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True), AddedToken(content="[PAD]", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True), AddedToken(content="[MASK]", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True)], limit_alphabet=None, initial_alphabet=[], continuing_subword_prefix=None, end_of_word_suffix=None, max_token_length=None, words={}))


In [ ]:
# optimization with pre-tokenizers
# pre-tokenizer to split input text into words based on whitespace
# optimizes the tokenization process by breaking down text into manageable chunks
# e.g "it is" can be a single token since those two words are often together in the training data, instead -> ["it", "is"]
# ensure that no token is bigger than a word
from tokenizers.pre_tokenizers import Split, Digits, Sequence
tokenizer.pre_tokenizer = Sequence([
    Split(pattern=r"\s+", behavior="isolated"), # split on whitespace, e.g. "it is" -> ["it", "is"], isolated means keep the split tokens, "it is" -> ["it", " ", "is"]
    Digits(individual_digits=False) # split digits from letters, e.g. "word123" -> ["word", "123"]
])

In [10]:
# train the tokenizer on each split of the WikiText-103 dataset
# relative fast training time even on large datasets
tokenizer.train_from_iterator(
	(row["text"] for split in ["train", "validation", "test"] for row in wiki_dataset[split]),
	trainer=trainer
)

In [11]:
tokenizer.save("./tmp/tokenizer-wiki.json")

In [31]:
tokenizer = Tokenizer.from_file("./tmp/tokenizer-wiki.json")

In [32]:
sentence = "Hello, devs😁! Welcome to the 1st AI crash course."
output = tokenizer.encode(sentence)
print("Tokens:", output.tokens)
print("IDs:", output.ids)

Tokens: ['Hell', 'o, ', 'dev', 's', '[UNK]', '!', ' W', 'el', 'com', 'e to the ', '1', 'st ', 'A', 'I ', 'crash ', 'course', '.']
IDs: [14653, 6242, 5598, 89, 0, 7, 8717, 5055, 5132, 5893, 23, 5223, 39, 5509, 16678, 22828, 20]


In [33]:
# offsets are also tracked to indicate the start and end positions of each token in the original input string
for i, token in enumerate(output.tokens):
    print(f"Token: {token}, Offsets: {output.offsets[i]}")
# discovered [UNK] emoji in the original string
sentence[11:12]

Token: Hell, Offsets: (0, 4)
Token: o, , Offsets: (4, 7)
Token: dev, Offsets: (7, 10)
Token: s, Offsets: (10, 11)
Token: [UNK], Offsets: (11, 12)
Token: !, Offsets: (12, 13)
Token:  W, Offsets: (13, 15)
Token: el, Offsets: (15, 17)
Token: com, Offsets: (17, 20)
Token: e to the , Offsets: (20, 29)
Token: 1, Offsets: (29, 30)
Token: st , Offsets: (30, 33)
Token: A, Offsets: (33, 34)
Token: I , Offsets: (34, 36)
Token: crash , Offsets: (36, 42)
Token: course, Offsets: (42, 48)
Token: ., Offsets: (48, 49)


'😁'

In [34]:
# check vocabulary size, same as trainer.vocab_size
print("Vocabulary Size:", tokenizer.get_vocab_size(), "trainer:", trainer.vocab_size)
# double check special tokens, defined during training
for token in ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]:
    token_id = tokenizer.token_to_id(token)
    print(f"Token: {token}, ID: {token_id}")


Vocabulary Size: 30000 trainer: 30000
Token: [UNK], ID: 0
Token: [CLS], ID: 1
Token: [SEP], ID: 2
Token: [PAD], ID: 3
Token: [MASK], ID: 4


In [35]:
# pairwise sequence tokenization with type IDs, e.g. for sentence pairs in tasks like QA or NLI (Natural Language Inference)
output = tokenizer.encode(sentence, "... second sentence.")
print(output.tokens)
print(output.type_ids)

['Hell', 'o, ', 'dev', 's', '[UNK]', '!', ' W', 'el', 'com', 'e to the ', '1', 'st ', 'A', 'I ', 'crash ', 'course', '.', '... ', 'second ', 'sent', 'ence', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [40]:
# decode
output = tokenizer.encode(sentence, "... second sentence.")
print(output.tokens)
print(output.type_ids)
decoded = tokenizer.decode(output.ids) #
print("🛑", decoded)

['Hell', 'o, ', 'dev', 's', '[UNK]', '!', ' W', 'el', 'com', 'e to the ', '1', 'st ', 'A', 'I ', 'crash ', 'course', '.', '... ', 'second ', 'sent', 'ence', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
🛑 Hell o,  dev s !  W el com e to the  1 st  A I  crash  course . ...  second  sent ence .


In [41]:
# decoder to properly handle BPE tokens
from tokenizers.decoders import BPEDecoder
tokenizer.decoder = BPEDecoder()
output = tokenizer.encode(sentence, "... second sentence.")
print(output.tokens)
print(output.type_ids)
# common pitfalls: keep in account special tokens during decoding
decoded = tokenizer.decode(output.ids, skip_special_tokens=False)
print("🛑", decoded)
decoded = tokenizer.decode(output.ids, skip_special_tokens=True)
print("✅", decoded)

['Hell', 'o, ', 'dev', 's', '[UNK]', '!', ' W', 'el', 'com', 'e to the ', '1', 'st ', 'A', 'I ', 'crash ', 'course', '.', '... ', 'second ', 'sent', 'ence', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
🛑 Hello, devs[UNK]! Welcome to the 1st AI crash course.... second sentence.
✅ Hello, devs! Welcome to the 1st AI crash course.... second sentence.


In [42]:
# apply post process, e.g. for BERT-like models
from tokenizers.processors import TemplateProcessing
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]", # for single sentence $A
    # pairs can be handled too, e.g. sentence similarity tasks
    pair="[CLS] $A [SEP] $B:1 [SEP]:1", # for pairs of sentences $A and $B, :1 is type IDs for second sentence (omitted in $A)
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")), # start of sequence
        ("[SEP]", tokenizer.token_to_id("[SEP]")), # end of sequence
    ],
)

In [43]:
output = tokenizer.encode(sentence, "... second sentence.")
print(output.tokens)
print(output.type_ids)
decoded = tokenizer.decode(output.ids, skip_special_tokens=True)
print("✅", decoded)

['[CLS]', 'Hell', 'o, ', 'dev', 's', '[UNK]', '!', ' W', 'el', 'com', 'e to the ', '1', 'st ', 'A', 'I ', 'crash ', 'course', '.', '[SEP]', '... ', 'second ', 'sent', 'ence', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
✅ Hello, devs! Welcome to the 1st AI crash course.... second sentence.


In [47]:
# speed up encoding with batch processing
sentences = [
    "Hello, devs 😁! Welcome to the 1st AI crash course.",
    "Tokenizers are really fast and efficient.",
    "We can process multiple sentences at once.",
    ["This is the fourth sentence.","Pair with another one  ."], #also pairs
    "Finally, the fifth sentence completes our batch."
    ]
outputs = tokenizer.encode_batch(sentences)
for output in outputs:
    print(output.tokens)
    print(output.type_ids)

['[CLS]', 'Hell', 'o, ', 'dev', 's ', '[UNK]', '!', ' W', 'el', 'com', 'e to the ', '1', 'st ', 'A', 'I ', 'crash ', 'course', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['[CLS]', 'T', 'ok', 'en', 'iz', 'ers are ', 'really ', 'fast ', 'and ', 'eff', 'icient', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['[CLS]', 'We ', 'can ', 'process ', 'multiple ', 'sent', 'ences ', 'at ', 'onc', 'e.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['[CLS]', 'This ', 'is the ', 'fourth ', 'sent', 'ence', '.', '[SEP]', 'P', 'air ', 'with ', 'another ', 'one ', ' ', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
['[CLS]', 'Fin', 'ally, the ', 'fifth ', 'sentence ', 'complet', 'es ', 'our ', 'bat', 'ch', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'

In [45]:
# enable padding: default to right, pad to the longest sequence in the batch
tokenizer.enable_padding(pad_id=tokenizer.token_to_id("[PAD]"), pad_token="[PAD]")
output = tokenizer.encode_batch(["Hello, y'all! Adding some context to create some padding", "How are you 😁 ?"]) # the pair sentence
print("Padded Tokens:", output[1].tokens)
print("Padded IDs:", output[1].ids)
# attention mask indicates which tokens should be attended to (1) and which are padding (0)
print(output[0].attention_mask)
print(output[1].attention_mask) # tokenizer takes the padding into account, e.g. skipping attention on padded tokens
tokenizer.save("./tmp/tokenizer-wiki-post.json")

Padded Tokens: ['[CLS]', 'H', 'ow ', 'are ', 'you ', '[UNK]', ' ', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Padded IDs: [1, 46, 5375, 5232, 7244, 0, 6, 37, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [54]:
# BPE vs pretrained WordPiece BERT cased/uncased
from tokenizers import Tokenizer
bpe = Tokenizer.from_file("./tmp/tokenizer-wiki-post.json")
bert_c = Tokenizer.from_pretrained("google-bert/bert-base-cased") 
bert_u = Tokenizer.from_pretrained("google-bert/bert-base-uncased")
for [tokenizer_name, tokenizer_instance] in [("BPE", bpe), ("BERT Cased", bert_c), ("BERT Uncased", bert_u)]:
    print(f"--- {tokenizer_name} ---")
    output = tokenizer_instance.encode(sentences[0])
    print(output.tokens)
    print(tokenizer_instance.decode(output.ids, skip_special_tokens=True))

--- BPE ---
['[CLS]', 'Hell', 'o, ', 'dev', 's ', '[UNK]', '!', ' W', 'el', 'com', 'e to the ', '1', 'st ', 'A', 'I ', 'crash ', 'course', '.', '[SEP]']
Hello, devs ! Welcome to the 1st AI crash course.
--- BERT Cased ---
['[CLS]', 'Hello', ',', 'de', '##v', '##s', '[UNK]', '!', 'Welcome', 'to', 'the', '1st', 'AI', 'crash', 'course', '.', '[SEP]']
Hello, devs! Welcome to the 1st AI crash course.
--- BERT Uncased ---
['[CLS]', 'hello', ',', 'dev', '##s', '[UNK]', '!', 'welcome', 'to', 'the', '1st', 'ai', 'crash', 'course', '.', '[SEP]']
hello, devs! welcome to the 1st ai crash course.


In [ ]:
# tokenizer card can expose a vocab
!wget -O ./tmp/bert-base-uncased-vocab.txt https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt

--2025-11-07 00:26:49--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.130.248, 16.15.178.253, 16.15.179.50, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.130.248|:443... connected.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘./tmp/bert-base-uncased-vocab.txt’

./tmp/bert-base-unc 100%[===================>] 226.08K   654KB/s    in 0.3s    

2025-11-07 00:26:50 (654 KB/s) - ‘./tmp/bert-base-uncased-vocab.txt’ saved [231508/231508]

